# Advent of Code Day 22

For Day 22, we need to simulate a viral infection.  We start with a grid of nodes that are either infected or clean and then the virus moves around the grid infecting or cleaning nodes (part one).  In part two, there are additional states of infection and the movements are different for each type.  The solution in this Notebook is only for part two since there wasn't an elegant way to model the two within one class. 

In [160]:
from utils import read_input
from enum import Enum

In [171]:
class InfectionStatus(Enum):
    WEAKENED = 1
    INFECTED = 2
    FLAGGED = 3

class Grid(object):
    
    def __init__(self, rows):
           
        if len(rows) != len(rows[0]):
            raise ValueError('rows must be 2D matrix with same dimensions')
        
        self.direction = 'U'
        self.infected = {}
        self.current_x = 0
        self.current_y = 0
        self.infection_count = 0
        
        center_index = int(round(len(rows) / 2))
        
        for row_index, row in enumerate(rows):
            for column_index, value in enumerate(row):
                if value == '#':
                    self.infected[(column_index - center_index, center_index - row_index)] = InfectionStatus.INFECTED
    
    def simulate(self, bursts):
        
        for _ in xrange(0, bursts):
            self.tick()
            
        
    def tick(self):
        self.change_direction()
        
        self.infect_or_clean()
        
        self.move()        
    
    
    def change_direction(self):
        
        # If the node we're on is currently clean (which we use to mean it's not in self.infected at all), 
        # we turn left
        if (self.current_x, self.current_y) not in self.infected:
            self.direction = self.turn_left()
            return
        
        current_status = self.infected[(self.current_x, self.current_y)]
        
        if current_status == InfectionStatus.WEAKENED:
            pass # if the current node is weakened, we don't turn at all
        elif current_status == InfectionStatus.INFECTED:
            self.direction = self.turn_right()
        elif current_status == InfectionStatus.FLAGGED:
            self.direction = self.turn_around()
        
            
    def infect_or_clean(self):
        
        current = (self.current_x, self.current_y)
        
        if current not in self.infected:
            self.infected[current] = InfectionStatus.WEAKENED
        elif self.infected[current] == InfectionStatus.WEAKENED:
            self.infected[current] = InfectionStatus.INFECTED
            self.infection_count += 1
        elif self.infected[current] == InfectionStatus.INFECTED:
            self.infected[current] = InfectionStatus.FLAGGED
        elif self.infected[current] == InfectionStatus.FLAGGED:
            del self.infected[current]
      
            
    def move(self):
        if self.direction == 'U':
            self.current_y += 1
        elif self.direction == 'D':
            self.current_y -= 1
        elif self.direction == 'L':
            self.current_x -= 1
        elif self.direction == 'R':
            self.current_x += 1
        else:
            raise ValueError('Not sure which direction to move')
        
    
    def turn_left(self):
        if self.direction == 'U':
            return 'L'
        elif self.direction == 'R':
            return 'U'
        elif self.direction == 'D':
            return 'R'
        elif self.direction == 'L':
            return 'D'
        else:
            raise ValueError('I can\'t figure out which way to turn')
        
    def turn_right(self):
        if self.direction == 'U':
            return 'R'
        elif self.direction == 'R':
            return 'D'
        elif self.direction == 'D':
            return 'L'
        elif self.direction == 'L':
            return 'U'
        else:
            raise ValueError('I can\'t figure out which way to turn')
            
    def turn_around(self):
        if self.direction == 'U':
            return 'D'
        elif self.direction == 'D':
            return 'U'
        elif self.direction == 'L':
            return 'R'
        elif self.direction == 'R':
            return 'L'
        else:
            raise ValueError('I can\'t figure out which way to turn')
    
    def display(self, bounding_box = None):
        
        if bounding_box == None:
            ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y)) = self._bounding_box()
        else:
            ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y)) = bounding_box
                   
        for y in xrange(top_left_y, bottom_right_y - 1, -1):
            
            row = []
         
            for x in xrange(top_left_x, bottom_right_x + 1):            
                
                if (x, y) not in self.infected:
                    ch = '.'
                elif self.infected[(x, y)] == InfectionStatus.WEAKENED:
                    ch = 'W'
                elif self.infected[(x, y)] == InfectionStatus.INFECTED:
                    ch = '#'
                elif self.infected[(x, y)] == InfectionStatus.FLAGGED:
                    ch = 'F'              
                row.append(ch)
            
            print ''.join(row)
        
         
    def _bounding_box(self):
        min_x = min([x for (x, y) in self.infected])
        max_x = max([x for (x, y) in self.infected])
        min_y = min([y for (x, y) in self.infected])
        max_y = max([y for (x, y) in self.infected])
        
        overall_min = min(min_x, min_y)
        overall_max = max(min_x, max_y)
                  
        return ((overall_min, overall_max), (overall_max, overall_min))
        
        
        


In [197]:
def solve_sample():
    initial = ['..#', '#..', '...']

    grid = Grid(initial)

    grid.display(((-4, 4), (4, -4)))

    grid.simulate(10000000)

    grid.infection_count

In [195]:
def solve_part_two():
    
    rows = read_input('Input/day22.txt')
    
    grid = Grid(rows)
    
    grid.simulate(10000000)
    
    print grid.infection_count
    

In [196]:
solve_part_two()

2511672
